In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from functools import reduce

## 1. Load and process ACS and county shapefile data

In [ ]:
# ID columns for study
cols = {'DP02_0002PE' : 'married',
        'DP02_0004PE' : 'cohab',
        'DP02_0016E'  : 'avg_hhsz', 
        'DP02_0068PE' : 'pct_educ',
        'DP02_0093PE' : 'pct_fb',
        'DP03_0009PE' : 'unempl',
        'DP03_0025E'  : 'mean_com',
        'DP03_0033PE' : 'pct_ag',
        'DP03_0035PE' : 'pct_manu',
        'DP03_0062E'  : 'med_inc',
        'DP03_0128PE' : 'pct_pov',
        'DP04_0047PE' : 'pct_rent',
        'DP04_0058PE' : 'pct_no_veh',
        'DP04_0089E'  : 'med_hval',
        'DP04_0134E'  : 'med_rent',
        'DP05_0001E'  : 'pop',
        'DP05_0019PE' : 'pct_un18',
        'DP05_0024PE' : 'pct_ov65',
        'DP05_0038PE' : 'pct_bl',
        'DP05_0044PE' : 'pct_as',
        'DP05_0071PE' : 'pct_hisp'}

In [ ]:
# suppress mixed data type warnings
import warnings
warnings.filterwarnings("ignore")

# load acs data
path = 'acs2019/acs2019_DP0'
#skip = [1]
dp02 = pd.read_csv(path + '2.csv',)# skiprows=skip)
dp03 = pd.read_csv(path + '3.csv',)# skiprows=skip)
dp04 = pd.read_csv(path + '4.csv',) #skiprows=skip)
dp05 = pd.read_csv(path + '5.csv',)# skiprows=skip)

In [ ]:
# merge all acs tables
acs_lst = [dp02, dp03, dp04, dp05]
acs = reduce(lambda left, right: pd.merge(left, right, on=['GEO_ID', 'NAME'], how='inner'), acs_lst)

# rename acs cols using dict
acs = acs.rename(columns=cols)

In [ ]:
# drop unneeded cols, remove weird first row, trim GEOID for shp merge, and remove duped cols
acs = acs[acs.columns.drop(list(acs.filter(regex='DP0')))]
acs = acs[1:]
acs['GEO_ID'] = acs['GEO_ID'].str.replace('0500000US', '')
#acs = acs.loc[:,~acs.columns.duplicated()]

In [ ]:
# merge w shp
ct = gpd.read_file('../668 final project/source_files/contig_us_county')
gdf = ct.merge(acs, how='inner', left_on='GEOID', right_on='GEO_ID', suffixes=('_SH',''))

# drop unneeded shp cols
gdf = gdf.drop(columns=['GEO_ID', 'LSAD', 'CLASSFP', 'MTFCC', 'CSAFP', 'METDIVFP',
                        'FUNCSTAT', 'INTPTLAT', 'INTPTLON', 'CBSAFP'])
gdf.plot()

In [ ]:
# create list of variable names, remove strings, convert vars to float
var = list(cols.values())
gdf[var] = gdf[var].replace(['N', '-'], np.nan)
gdf[var] = gdf[var].astype('float')

# verify (false is good)
(gdf[var].dtypes == 'O').any()

In [ ]:
# create state column and drop leftover col
gdf[['NAME', 'STATE']] = gdf['NAME'].str.split(', ', expand=True)
gdf = gdf.drop(columns=['NAMELSAD'])

In [ ]:
# add state abbrev, region, and division cols (drop )
url = 'https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv'
reg = pd.read_csv(url, encoding = 'ISO-8859-1', delimiter=',')#, index_col='st')
gdf = gdf.merge(reg, how='left', left_on='STATE', right_on='State')
gdf = gdf.drop(columns=['STATE'])

## 2. Preliminary analysis

In [ ]:
# add pop density column (pop/km) and append to vars list (if not already in there)
gdf['pop_dens'] = gdf['pop'] / (gdf['ALAND'] / 1000)

if 'pop_dens' in var:
    pass
else:
    var.append('pop_dens')

In [ ]:
# product summary stats table for all counties
for x in [var]:
    summ = pd.DataFrame(gdf[x].describe().round(2)).T
summ

In [ ]:
# summary stats table by Region
gdf.groupby('Region')[var].mean().round(2).T

In [ ]:
# plot US divisions
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax,
         column='Division',
         edgecolor='white',
         linewidth=0.1)

plt.title(label='US FIPS Divisions')
ax.set_axis_off()

In [ ]:
# plot cohabitation rate
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax,
         column='cohab',
         cmap='viridis',
         edgecolor='white',
         linewidth=0.1,
         legend=True,
         legend_kwds={'shrink': 0.5})

plt.title(label='Cohabitating Household Percentage')
ax.set_axis_off()

In [ ]:
# investigate the clear outlier
gdf[gdf['cohab'] > 13].sort_values(by='cohab', ascending=False)

In [ ]:
# plot marriage rates
fig, ax = plt.subplots(figsize=(10, 10))
g = gdf.plot(ax=ax,
             column='married',
             cmap='viridis',
             edgecolor='white',
             linewidth=0.1,
             legend=True,
             legend_kwds={'shrink': 0.5})

plt.title(label='Married Household Percentage')
ax.set_axis_off()

In [ ]:
# plot log pop (and pop dens)
gdf['log_pop'] = np.log(gdf['pop'])
gdf['log_pop_dens'] = np.log(gdf['pop_dens'])

graph = 'log_pop'
fig, ax = plt.subplots(figsize=(10, 10))
g = gdf.plot(ax=ax,
             column=graph,
             cmap='viridis',
             edgecolor='white',
             linewidth=0.1,
             legend=True,
             legend_kwds={'shrink': 0.5})

plt.title(label=graph)
ax.set_axis_off()